In [1]:
# built-in
import os
import sys

module_path = os.path.abspath(os.path.join('..')) # or the path to your source code
sys.path.insert(0, module_path)
module_path = os.path.abspath(os.path.join('../respiratoryanalysis')) # or the path to your source code
sys.path.insert(0, module_path)

# third-party
import pandas as pd
import numpy as np
import pickle

# local
from respiratoryanalysis.waveform import get_waveforms_by_id_by_activity
from respiratoryanalysis.get_data import get_data_by_id_activity
from respiratoryanalysis.constants import DEVICE_INFO

In [2]:
with open('../results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

acquisition_folderpath = '../Aquisicao'
data, _ = get_data_by_id_activity(acquisition_folderpath)

Getting data for participants...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [3]:
amplitudes_df = pd.DataFrame(columns=['amplitude', 'activity', 'sensor', 'id'])

for id in overview.keys():
    for activity in overview[id].keys():
        for device in ['mag', 'pzt', 'airflow']:
            waveforms,_ = get_waveforms_by_id_by_activity(data[id][activity][device].values, overview[id][activity][DEVICE_INFO[device]["name"]])
            x = np.amax(waveforms, axis=1) - np.amin(waveforms, axis=1)
            amplitudes_df = pd.concat([amplitudes_df, pd.DataFrame({'amplitude': x, 'activity': activity, 'sensor': DEVICE_INFO[device]["sensor"], 'id': id})], ignore_index=True)

amplitudes_df            

amplitude activity   sensor    id
0       40.535129      SNB      MAG  7OYX
1       27.815240      SNB      MAG  7OYX
2       40.000312      SNB      MAG  7OYX
3       46.846376      SNB      MAG  7OYX
4       44.401724      SNB      MAG  7OYX
...           ...      ...      ...   ...
5959  1641.079239       SS      PZT  D4GQ
5960  1563.995320       SS  Airflow  D4GQ
5961  2062.371732       SS  Airflow  D4GQ
5962  1431.444723       SS  Airflow  D4GQ
5963  3029.187856       SS  Airflow  D4GQ

[5964 rows x 4 columns]

In [4]:
amplitudes_df[amplitudes_df["sensor"]=='MAG'].max()

amplitude    330.553946
activity            UAR
sensor              MAG
id                 Y6O3
dtype: object

In [5]:
print(f"pk-to-pk for MAG sensor: {np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']):.1f} +/- {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']):.1f}")
print(f"pk-to-pk for MAG sensor: {np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']):.1f} +/- {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']):.1f}")
print(f"pk-to-pk for Airflow sensor: {np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']):.1f} +/- {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']):.1f}\n")

print(f"coefficient of variation MAG: {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']) / np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='MAG']):.2f}")
print(f"coefficient of variation PZT: {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']) / np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='PZT']):.2f}")
print(f"coefficient of variation Airflow: {np.std(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']) / np.mean(amplitudes_df['amplitude'][amplitudes_df['sensor']=='Airflow']):.2f}")

pk-to-pk for MAG sensor: 56.8 +/- 32.3
pk-to-pk for MAG sensor: 1154.2 +/- 1007.3
pk-to-pk for Airflow sensor: 1165.7 +/- 461.7

coefficient of variation MAG: 0.57
coefficient of variation PZT: 0.87
coefficient of variation Airflow: 0.40


In [6]:
amplitudes_by_activity_df = pd.DataFrame(columns=['Activity', 'Sensor', 'Pk-to-pk amplitude', 'std/mean'])

for activity in amplitudes_df['activity'].unique():
    amps = amplitudes_df[(amplitudes_df['activity'] == activity) & (amplitudes_df['sensor'] == 'MAG')]['amplitude'].values
    mean_amp = np.mean(amps)
    std_amp = np.std(amps)
    amplitudes_by_activity_df = pd.concat([amplitudes_by_activity_df, pd.DataFrame({'Activity': activity, 'Sensor': 'MAG', 'Pk-to-pk amplitude': f'${np.around(mean_amp, 2)} \pm {np.around(std_amp)}$', 'std/mean': np.round(std_amp/mean_amp, 2)}, index=[0])], ignore_index=True)
    
    amps = amplitudes_df[(amplitudes_df['activity'] == activity) & (amplitudes_df['sensor'] == 'PZT')]['amplitude'].values
    mean_amp = np.mean(amps)
    std_amp = np.std(amps)
    amplitudes_by_activity_df = pd.concat([amplitudes_by_activity_df, pd.DataFrame({'Activity': activity, 'Sensor': 'PZT', 'Pk-to-pk amplitude': f'${np.around(mean_amp, 2)} \pm {np.around(std_amp)}$', 'std/mean': np.round(std_amp/mean_amp, 2)}, index=[0])], ignore_index=True)

amplitudes_by_activity_df

Activity Sensor    Pk-to-pk amplitude  std/mean
0       SNB    MAG      $35.68 \pm 14.0$      0.41
1       SNB    PZT    $936.36 \pm 669.0$      0.71
2       SGB    MAG      $75.28 \pm 37.0$      0.49
3       SGB    PZT   $1398.26 \pm 921.0$      0.66
4      MIXB    MAG      $50.76 \pm 28.0$      0.55
5      MIXB    PZT   $1277.67 \pm 862.0$      0.68
6      STNB    MAG       $41.5 \pm 22.0$      0.53
7      STNB    PZT   $1100.16 \pm 686.0$      0.62
8       MCH    MAG       $57.9 \pm 26.0$      0.45
9       MCH    PZT   $1289.49 \pm 773.0$      0.60
10      SQT    MAG      $72.34 \pm 38.0$      0.53
11      SQT    PZT   $1664.49 \pm 975.0$      0.59
12      AAL    MAG      $49.69 \pm 21.0$      0.43
13      AAL    PZT   $1046.24 \pm 977.0$      0.93
14      AAR    MAG      $49.61 \pm 22.0$      0.43
15      AAR    PZT   $1039.32 \pm 982.0$      0.94
16      ALL    MAG       $59.7 \pm 25.0$      0.41
17      ALL    PZT  $1200.24 \pm 1034.0$      0.86
18      ALR    MAG      $50.37 \pm 26.0$      0.52
19      ALR    PZT   $1101.37 \pm 880.0$      0.80
20      UAL    MAG      $61.29 \pm 31.0$      0.51
21      UAL    PZT   $898.73 \pm 1022.0$      1.14
22      UAR    MAG      $54.64 \pm 32.0$      0.58
23      UAR    PZT    $642.03 \pm 919.0$      1.43
24       SE    MAG       $55.1 \pm 26.0$      0.47
25       SE    PZT   $800.31 \pm 1024.0$      1.28
26       SS    MAG      $63.37 \pm 46.0$      0.73
27       SS    PZT  $1296.89 \pm 1188.0$      0.92
28       TR    MAG       $76.1 \pm 43.0$      0.56
29       TR    PZT  $1679.46 \pm 1162.0$      0.69

In [7]:
amplitudes_df

amplitude activity   sensor    id
0       40.535129      SNB      MAG  7OYX
1       27.815240      SNB      MAG  7OYX
2       40.000312      SNB      MAG  7OYX
3       46.846376      SNB      MAG  7OYX
4       44.401724      SNB      MAG  7OYX
...           ...      ...      ...   ...
5959  1641.079239       SS      PZT  D4GQ
5960  1563.995320       SS  Airflow  D4GQ
5961  2062.371732       SS  Airflow  D4GQ
5962  1431.444723       SS  Airflow  D4GQ
5963  3029.187856       SS  Airflow  D4GQ

[5964 rows x 4 columns]

### Analysis on alternative positioning

In [8]:
amplitudes_middle_df = pd.DataFrame(columns=['amplitude', 'activity', 'sensor', 'id'])

for id in overview_middle.keys():
    for activity in overview_middle[id].keys():
        for device in ['mag', 'pzt']:
            waveforms,_ = get_waveforms_by_id_by_activity(data[id][activity][device].values, overview_middle[id][activity][DEVICE_INFO[device]["name"]])
            x = np.amax(waveforms, axis=1) - np.amin(waveforms, axis=1)
            amplitudes_middle_df = pd.concat([amplitudes_middle_df, pd.DataFrame({'amplitude': x, 'activity': activity, 'sensor': DEVICE_INFO[device]["sensor"], 'id': id})], ignore_index=True)

amplitudes_middle_df            

amplitude activity sensor    id
0       27.254948     SNBm    MAG  7OYX
1       23.778796     SNBm    MAG  7OYX
2       30.643432     SNBm    MAG  7OYX
3       37.540827     SNBm    MAG  7OYX
4       30.851934     SNBm    MAG  7OYX
...           ...      ...    ...   ...
1058   934.674910     UARm    PZT  D4GQ
1059  1774.597229     UARm    PZT  D4GQ
1060   400.716087     UARm    PZT  D4GQ
1061   550.935492     UARm    PZT  D4GQ
1062   703.998083     UARm    PZT  D4GQ

[1063 rows x 4 columns]

In [9]:
print(f"pk-to-pk for MAG sensor: {np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG'])} +/- {np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG'])}")
print(f"pk-to-pk for PZT sensor: {np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='PZT'])} +/- {np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='PZT'])}")
print(f"pk-to-pf for Airflow: {np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='Airflow'])} +/- {np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='Airflow'])}")
print(np.std(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG']) / np.mean(amplitudes_middle_df['amplitude'][amplitudes_middle_df['sensor']=='MAG']))

pk-to-pk for MAG sensor: 60.781023924237736 +/- 34.99986268975963
pk-to-pk for PZT sensor: 852.249766619987 +/- 848.9895225937855
pk-to-pf for Airflow: nan +/- nan
0.5758353583083138
